In [1]:
import pandas as pd

ratings_path = "datas/ratings.csv"
ratings = pd.read_csv(ratings_path)


user_counts = ratings['userId'].value_counts()

# Filtrer les utilisateurs ayant au moins 50 lignes
users_to_keep = user_counts[user_counts >= 50].index
user_filtered_df = ratings[ratings['userId'].isin(users_to_keep)]


user_filtered_df.to_csv('datas/ratings_filtered_v1.csv', index=False)
print(f"Les utilisateurs ayant moins de 10 notes ont été supprimées")

La taille du tableau initial est : (32000204, 4)
Les utilisateurs ayant moins de 10 notes ont été supprimées


In [2]:
movie_counts = user_filtered_df['movieId'].value_counts()

# Filtrer les films ayant au moins 10 lignes
movies_to_keep = movie_counts[movie_counts >= 10].index
movie_filtered_df = user_filtered_df[user_filtered_df['movieId'].isin(movies_to_keep)]

movie_filtered_df.to_csv('datas/ratings_filtered_vf.csv', index=False)

print("Les films ayant moins de 10 notes ont été supprimées")



Les films ayant moins de 10 notes ont été supprimées


In [3]:
movies = pd.read_csv("datas/movies.csv")

merged_table = pd.merge(movie_filtered_df, movies, on='movieId', how='left')

merged_table.to_csv("datas/ratings_with_genres.csv", index=False)

print("Les genres ont été fusionnés avec les notes")

Les genres ont été fusionnés avec les notes


In [4]:
# Séparer les genres dans des lignes distinctes
ratings_with_genre_separated = merged_table.assign(genres=merged_table['genres'].str.split('|')).explode('genres')

ratings_with_genre_separated.to_csv('datas/ratings_with_genre_separated.csv', index=False)

In [5]:
# Création du csv contentant les notes par genre des utilisateurs
average_ratings = ratings_with_genre_separated.groupby(['userId', 'genres'])['rating'].mean().reset_index()
average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)

global_genre_mean = average_ratings.groupby('genres')['average_rating'].mean()

# Transformer le tableau pour que les genres deviennent des colonnes
pivot_table = average_ratings.pivot(index='userId', columns='genres', values='average_rating')

# Remplacer les valeurs manquantes par des 0
pivot_table = pivot_table.fillna(0)
pivot_table.to_csv('datas/user_genre_ratings_with_0.csv', index=False)